# Lab 3 - Arend Wong (aw3146) - 2021-05-29

<h4>

    Qs:
    1. Run a simple bivariate regression, and interpret your results. (Did the results fit your expectations? Why? Why not?)
    
    2. Add an additional variable that might mediate or partly "explain" the initial association from that simple regression above -- and explain your results. Did it work out? Yes? No?
    
    3. Run another multiple regression. Tell me how you expect your dependent variable to be affected by the independent variables. Interpret your results.
    
    4. Now add another independent variable to that model in Question 3, specifically a set of dummy variables. Tell me why you added that new set of variables and what effect you expected them to have. Did they have an effect? Interpret that new model. 
    
    
</h4>

In [5]:
# importing relevant modules...

from __future__ import division
import pandas as pd
import numpy as np
import re, os
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns #os.system('pip install seaborn')
from scipy import stats
import pycountry_convert as pc #os.system("pip install pycountry-convert")
from colour import Color #os.system('pip install colour')

# defining helper functions for later in my code

def printurn(x):
    print(x)
    return(x)

def recode_onehot_if(s='CHILDRENS COAT', pattern='CHILDREN'):
    result = '1' if pattern in str(s) else '0'
    return(result)

def check_color(color):
    try:
        Color(color)
        return(True)
    except ValueError as e:
        return(False)

def extract_color(pdcolumn):
    pdcolumn = pd.Series(pdcolumn.copy()) if type(pdcolumn)==str else pd.Series(pdcolumn) if type(pdcolumn)==list else pdcolumn
    newcolumn = pdcolumn.copy().apply(lambda s: ' '.join([i for i in str(s).split(' ') if check_color(i)]))
    return(newcolumn)

def compute_percentage(x, my_crosstab):
      pct = float(x/my_crosstab['count'].sum()) * 100
      return round(pct, 2)

# os.listdir('../lab1')

### Reading dataframe! 
#### Description of dataset chosen:
Reading data below! We're using our own custom dataset (not the GSS data!). I found this sample retail sales dataset at this link: #https://www.kaggle.com/carrie1/ecommerce-data. I'm using this dataset because it aligns with my personal research interests and I wanted to make this lab as utilitarian as possible! It's data from a UK-based retailer. Each observation (row) of the data corresponds to one sale. The data has price of item, quantity sold in each individual purchase transaction, country of sale, and description of item. I'm focusing on the description of the items and recoding that text variable into more useful categorical variables (explained below).

In [6]:
try: # NOTE: trying to read the csv file with basic default args first, but if error, we're adding the encoding argument. Since this dataset has text data, the encoding has some quirks and some Python & Pandas versions don't support it with default UTF-8 encoding.
    df = pd.read_csv('../lab1/data.csv').drop_duplicates().query('Quantity>=0 and Quantity<=70000 and UnitPrice>=0').reset_index(drop=True) # dataset is from #https://www.kaggle.com/carrie1/ecommerce-data
except:
    df = pd.read_csv('../lab1/data.csv', encoding="ISO-8859-1").drop_duplicates().query('Quantity>=0 and Quantity<=1000 and UnitPrice>=0').reset_index(drop=True) #https://www.kaggle.com/carrie1/ecommerce-data 
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
526045,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
526046,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
526047,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
526048,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


### Recoding new variables, derived from current ones...

- invoice_month: nominal-ish. I'm creating a new variable called invoice_month from the invoice_date. I'm extracting just the month from the whole date codes in the InvoiceDate variable. It's nominal-ish because each class theoretically belongs to a consecutive month category, ie: January=1, Feb=2, Mar=3, Apr=4, ..., Nov=11, Dec=12. This variable will let us examine trends as the year progresses, but it's making the analysis simpler for us because it's ideologically binning continuous time into nominal months.
- n_words_description: continuous. I'm creating a new variable called n_words_description from the Description columns (description of each retail item). I'm extracting just the number of words in the description. I'm wondering if longer/more verbose descriptions may be associated with any sales trends. By extracting the number of words in each description, we have a simple but useful heuristic for verbosity of item-description.
- Additional recoding (not totally relevant to the Qs in the lab): - material and continent
- I'm recoding some variables/creating new ones below! I'm also filtering the data so that it only has the observations we care about (valid UnitPrices and valid Quantity sold for each given transation). My new material categorical variable is derived from the Description column. I pulled out possible materials from the descriptions of the items sold. I'll summarize them below.
- I'm recoding a new variable called continent from the current Country variable. This will help us get a narrowed down data set of georaphies so as to really analyze the material purchased based off geographical region. I'm recoding the country variable as a new variable (continent for example) by using a python package called pycountry_convert and looping through each class in our Country variable.
- For curiority's sake, I created a few other categorical variables from the description column too (but I won't focus on them for this lab). I extracted item color, base item object, and whether the items is meant for children or not. All of these derived variables are first-drafts just for identifying initial patterns in the data. They're not perfect, but they're helpful for this exercise!

In [7]:
df_new = df.copy().apply(lambda y: y.str.lower() if str(y.dtype)=='object' else y).assign(
    purchase_price = lambda d: d.UnitPrice * d.Quantity,
    invoice_month = lambda d: pd.DatetimeIndex(d.InvoiceDate).month,
    invoice_year = lambda d: pd.DatetimeIndex(d.InvoiceDate).year,
    invoice_day_of_month = lambda d: pd.DatetimeIndex(d.InvoiceDate).day,
    n_words_description = lambda d: [len(str(x).split()) for x in d.Description],
    material=lambda d: d.Description.str.extract(r'(ceramic|metal|wood|plastic|cloth|felt|fabric|silk|polyester|linen|cotton|compostable|mahogany|canvas|cashmere|chiffon|denim|viscose|wool|fur|lace|leather|diamond|crystal|rhinestone|jewel|birch|rubber|wax|vintage|organic|pashmina|satin|spandex|suede|cement|marble)', expand=False, flags=re.IGNORECASE),
    color=lambda d: extract_color(d.Description).apply(lambda s: s.split(' ')[0]),
    ).apply(lambda y: y.str.lower().str.strip().replace('', np.nan) if str(y.dtype)=='object' else y)
print('df_new.shape # (531283, 11)', df_new.shape) # (531283, 11))
new_list = []
for boob in df_new.Country.str.title():
    try:
        new_boob = pc.country_name_to_country_alpha2(boob, cn_name_format="default")
        new_boob = pc.country_alpha2_to_continent_code(new_boob)
    except:
        new_boob = ''
    new_list.append(new_boob)
df_new['continent'] = new_list
df_new


df_new.shape # (531283, 11) (526050, 15)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,purchase_price,invoice_month,invoice_year,invoice_day_of_month,n_words_description,material,color,continent
0,536365,85123a,white hanging heart t-light holder,6,12/1/2010 8:26,2.55,17850.0,united kingdom,15.30,12,2010,1,5,NaN,white,EU
1,536365,71053,white metal lantern,6,12/1/2010 8:26,3.39,17850.0,united kingdom,20.34,12,2010,1,3,metal,white,EU
2,536365,84406b,cream cupid hearts coat hanger,8,12/1/2010 8:26,2.75,17850.0,united kingdom,22.00,12,2010,1,5,NaN,NaN,EU
3,536365,84029g,knitted union flag hot water bottle,6,12/1/2010 8:26,3.39,17850.0,united kingdom,20.34,12,2010,1,6,NaN,NaN,EU
4,536365,84029e,red woolly hottie white heart.,6,12/1/2010 8:26,3.39,17850.0,united kingdom,20.34,12,2010,1,5,wool,red,EU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526045,581587,22613,pack of 20 spaceboy napkins,12,12/9/2011 12:50,0.85,12680.0,france,10.20,12,2011,9,5,NaN,NaN,EU
526046,581587,22899,children's apron dolly girl,6,12/9/2011 12:50,2.10,12680.0,france,12.60,12,2011,9,4,NaN,NaN,EU
526047,581587,23254,childrens cutlery dolly girl,4,12/9/2011 12:50,4.15,12680.0,france,16.60,12,2011,9,4,NaN,NaN,EU
526048,581587,23255,childrens cutlery circus parade,4,12/9/2011 12:50,4.15,12680.0,france,16.60,12,2011,9,4,NaN,NaN,EU


#### Value counts of the words in our Description column, to give us inspiration on what possible dummy variables we might want to create or ways we could one-hot-encode different descriptive characteristics of each purchase observation.

In [8]:
print(df_new.material.value_counts().head())
# Checking value counts (frequencies) of words in the description field
pd.Series(' '.join(df_new.Description.astype(str).unique().tolist()).split()).value_counts()#.head(20)


vintage    32998
wood       25234
metal      20939
felt        9589
ceramic     8770
Name: material, dtype: int64


set               322
pink              298
of                240
heart             236
vintage           219
                 ... 
mousey              1
neckl.36"green      1
giraffe             1
mitt                1
confusing           1
Length: 2422, dtype: int64

### Creating a new column called `bulk_item_indicator`, which signifies whether an item comes in a multi-pack or not. For example, a 10-pack of toilet paper would be one item, but we've consider it a bulk item because it has 10 subunits/subitems in the one bundle. We're defining the classes of `bulk_item_indicator` as Yes or No, based on whether the description of the retail item purchased has keywords that indicate it's a bulk multipack. We're pulling these out using a regular expression. If the given `Description` string has words such as `set`, `pack`, `box of`, or `crate of`, then we're considering it a `Yes` in terms of the `bulk_item_indicator`.

In [9]:
# df_new.query('material=="vintage"')
df_new['bulk_item_indicator'] = np.where(df_new.Description.astype(str).str.contains('\\b(set|pack|box of|crate of)\\b'), 'Yes', 'No')

/Users/rhoadss/.pyenv/versions/3.8.7/lib/python3.8/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


<h2><b style="color: red">
    1. Run a simple bivariate regression, and interpret your results. (Did the results fit your expectations? Why? Why not?)
</b></h2>

### - We're running a simple regression exploring the relationship between `bulk_item_indicator` (independent variable) and `UnitPrice` (dependent variable). I expect that items sold in multipacks (`bulk_item_indicator==Yes`) will have higher UnitPrices because logically, packs of multiple items tend to cost more than items that don't come in multipacks. I'll use the toilet paper example again. A 10-pack of toilet paper (bulk) would logically be more expensive than a single roll of toilet paper would be, even though the ratio of price to subunit (roll) is probably lower for the bulk-pack. Sadly, we don't have a way to control for whether different retail items in our dataset come in different sized sets. Therefore, we'll likely see confounds/noise in our results. 

In [96]:
br_fashion1 = smf.ols(formula = 'UnitPrice~bulk_item_indicator', data = df_new).fit()
print(br_fashion1.summary())

                            OLS Regression Results                            
Dep. Variable:              UnitPrice   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     60.39
Date:                Sun, 30 May 2021   Prob (F-statistic):           7.79e-15
Time:                        14:55:12   Log-Likelihood:            -2.6318e+06
No. Observations:              525937   AIC:                         5.264e+06
Df Residuals:                  525935   BIC:                         5.264e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

## Interpretation: 
### - the bivariate regression does NOT match our expectations because (oddly) bulk items cost statistically significantly less on average than items that are not bulk/multipacks, with a p-value of 0.000, showcasing that the regression is statistically significant. There is a negative relationship between UnitPrice and bulk_item_indicator, such that when bulk_item_indicator is yes, the price is lower by $1.07. (p<0.001, coef=-1.0747). 
### - These unexpected results are probably related to the confounding/unmeasured predictors that I referenced in my hypothesis above. Perhaps some items that are highly expensive and never really sold in bulk directly to consumers are skewing our results (like diamonds). We might be able to measure this with the proxy variable of Quantity.

<h2><b style="color: red">
    2. Add an additional variable that might mediate or partly "explain" the initial association from that simple regression above -- and explain your results. Did it work out? Yes? No?
</b></h2>

### - Adding the `Quantity` variable to see if it mediates our relationship between `bulk_item_indicator` and `UnitPrice`. Items that already come in multi-packs are probably only purchased in lower quantities because one bulk pack of many items is probably enough. Quantity of units purchased is the mediator for the relationship between whether multipack items are higher or lower price.

In [11]:
br_fashion2 = smf.ols(formula = 'UnitPrice~bulk_item_indicator+Quantity', data = df_new).fit()
print(br_fashion2.summary())

                            OLS Regression Results                            
Dep. Variable:              UnitPrice   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     77.45
Date:                Sun, 30 May 2021   Prob (F-statistic):           2.34e-34
Time:                        22:35:26   Log-Likelihood:            -2.6322e+06
No. Observations:              526050   AIC:                         5.264e+06
Df Residuals:                  526047   BIC:                         5.265e+06
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [12]:
-0.0111 - -1.0731

1.0619999999999998

### - The Quantity variable changes the sign (pos/neg) of the original variable's coefficient, so it seems to be mediating the effect. Net of whether an item is a bulk/multipack item, for each additional Quantity (item) purchased in one transaction, the UnitPrice is $1.06 higher. This is quite a confusing relationship. I still feel like we're not capturing some important pieces of the puzzle, since we don't have any better proxy variables to help us explain the odd effect of bulk_item_indicator on UnitPrice.

### - Some further thoughts: From the results above, it contradicted our initial hypothesis as items that came in multi-packs, as established by bulk_item_indicator, was purchased in higher quantities despite the unit price of multipack items being higher. 


<h2><b style="color: red">
    3. Run another multiple regression. Tell me how you expect your dependent variable to be affected by the independent variables. Interpret your results.
</b></h2>

### - Perhaps invoice month predicts quantity of items purchased, but controlling for whether the item comes in a multipack (bulk_item_indicator) because an item being a multipack might confound the relationship between invoice month and Quantity. I'm thinking invoice month might be related to quantity because people might make more purchases later in the year where all the big gifting holidays happen.


In [21]:
br_fashion3 = smf.ols(formula = 'Quantity~invoice_month+bulk_item_indicator', data = df_new).fit()
print(br_fashion3.summary())

                            OLS Regression Results                            
Dep. Variable:         purchase_price   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     38.04
Date:                Sun, 30 May 2021   Prob (F-statistic):           3.01e-17
Time:                        23:17:08   Log-Likelihood:            -3.1229e+06
No. Observations:              526050   AIC:                         6.246e+06
Df Residuals:                  526047   BIC:                         6.246e+06
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

### Oddly, as the invoice month increases, the Quantity of the given item purchased is lower by about .09 units on average, statistically significantly (p<.001). This regression is notable because the addition of the invoice_month variable nullified the effect of bulk_item_indicator on Quantity. It's not non-significant (p=.407). In the future, it'd be useful to delve further into this phenomenon and see what it is about invoice month that impacts Quantity of items purchased when we're controlling for the bulk_item_indicator class. Maybe the month predictor isn't totally linear. Maybe prices go up and down as the year progresses, in a different type of interval pattern.

<h2><b style="color: red">
    4. Now add another independent variable to that model in Question 3, specifically a set of dummy variables. Tell me why you added that new set of variables and what effect you expected them to have. Did they have an effect? Interpret that new model. 
</b></h2>

### - I want to see if the geographic location of the item purchased has any impact on the relationship between bulk/multipack item classification and their unit Quantity of purchase. Maybe some countries have different tariff laws on certain types of goods that others don't have. Further continents might be harder to ship to at all, so they wouldn't want to ship too many of each item. For the sake of brevity, I'm going to use my custom-encoded continent variable, as opposed to the original Country variable, because Country just has far too many classes to analyze for the purpose of this lab. Thus, we should note that I might be making unfair generalizations about countries in the same continents having similar tariff laws or customs. Like I alluded to, maybe shipping costs end up factoring into the Quantities, which could suggest that customers far from the retailer's UK location might have to pay more for the same products, so they don't buy them at such high quantities.
### - Below, I'm recoding my continent as a set of dummy variables (one for each continent class). EU=Europe, AS=Asia, OC=Oceania, SA=South America, NA=North America, unknown=Unknown customer location

In [15]:
print(df_new.continent.value_counts())

continents_list = ['EU', 'AS', 'OC', 'SA', 'NA', '']
for each_continent_code in continents_list:
    df_new['continent_' + re.sub('^$', 'unknown', each_continent_code)] = np.where(df_new.continent==each_continent_code, 1, 0)
    
df_new.iloc[:,15:]

EU    513456
        9369
AS      1858
OC      1184
NA       151
SA        32
Name: continent, dtype: int64


,continent,bulk_item_indicator,continent_EU,continent_AS,continent_OC,continent_SA,continent_NA,continent_unknown
0,EU,No,1,0,0,0,0,0
1,EU,No,1,0,0,0,0,0
2,EU,No,1,0,0,0,0,0
3,EU,No,1,0,0,0,0,0
4,EU,No,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...
526045,EU,Yes,1,0,0,0,0,0
526046,EU,No,1,0,0,0,0,0
526047,EU,No,1,0,0,0,0,0
526048,EU,No,1,0,0,0,0,0


In [23]:
br_fashion4 = smf.ols(formula = 'Quantity~invoice_month+bulk_item_indicator+continent_AS+continent_EU+continent_OC+continent_NA+continent_SA+continent_unknown', data = df_new).fit()
print(br_fashion4.summary())

                            OLS Regression Results                            
Dep. Variable:               Quantity   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     405.4
Date:                Sun, 30 May 2021   Prob (F-statistic):               0.00
Time:                        23:18:37   Log-Likelihood:            -2.7308e+06
No. Observations:              526050   AIC:                         5.462e+06
Df Residuals:                  526042   BIC:                         5.462e+06
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

### It seems like item Quantities per purchase for Oceania have the most alarming results: they're statistically significantly much higher than they are for the other continents. This is contrary to my hypothesis. Maybe places like Oceania actually like to buy high quantities of items at once in order to stock up a lot since they probably won't want to do another super far shipment again later. Oceania is far and rather remote, with small scattered islands. (\$49.13 more than when the continent isn't OC; p<.001)

### - When adding the continent dummy variables to the model, our bulk_item_indicator predictor no longer has a statistically significant impact on Quantity of items per purchase. (p=.824; coef=.037), meaning bulk/multipack items don't seem to be purchased at different quantities than non-bulk items. This also makes my Oceania, far-away continent idea crumble a bit.
### - The rest of the continent dummies have unexpected results too: Asia (which is far from the UK) also sees higher Quantities per unit in transaction on average (4.13), regardless of whether it's a bulk item or not, and regardless of purchase-month. The Americas also see some lower Quantities of items per purchase, but NOT statistically significantly.
### - Unforeseen, the EU has stat. signif lower Quantities of item per purchase than other continents (by 11.78; p<.001). This might make sense because the company is in the EU, so selling to EU customers is likely cheaper/easier to deal with more transactions and customers probably come back a lot to buy more instead of stocking up super heavy in one sitting. Maybe there are also a variety of trade-related + logistical reasons. However, when controlling for the continent of purchase, the higher the purchase month (the later it is in the year) the lower is the Quantity of items per purchase on average by .07 (p<.001).